# 作業 : (Kaggle)鐵達尼生存預測

In [32]:
# 載入套件與資料
import pandas as pd
import numpy as np

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']

df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)  # 丟除'PassengerId', 'Survived'
# df_train.shape (891, 10)
df_test = df_test.drop(['PassengerId'] , axis=1)  # 丟除'PassengerId'
# df_test.shape  (418, 10)

df = pd.concat([df_train,df_test])

df #1309 rows × 10 columns

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [36]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [37]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [58]:
# 例 : 整數 (int) 特徵取平均 (mean)
print(df[int_features].mean())
print("-----------------------------")
print(df[int_features].max())
print("-----------------------------")
print(df[int_features].nunique())

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64
-----------------------------
Pclass    3
SibSp     8
Parch     9
dtype: int64
-----------------------------
Pclass    3
SibSp     7
Parch     8
dtype: int64


In [57]:
print(df[float_features].mean())
print("-----------------------------")
print(df[float_features].max())
print("-----------------------------")
print(df[float_features].nunique())

Age     29.881138
Fare    33.295479
dtype: float64
-----------------------------
Age      80.0000
Fare    512.3292
dtype: float64
-----------------------------
Age      98
Fare    281
dtype: int64


In [56]:
print(df[object_features].mean())  # ans: object no mean, Series([], dtype: float64)
print("-----------------------------")
print(df[object_features].max())  # ans: 字串無法分出最大值
print("-----------------------------")
print(df[object_features].nunique())

Series([], dtype: float64)
-----------------------------
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object
-----------------------------
Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64


## 作業2
Ans: object 類型,需先判斷是否許於類別型態,判斷有序、無序問題,無序的等價關係就使用One-hot encoding.有序的類別資料通常是直接使用數值替換.